<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Robust Regression

_Authors: Greg Baker (SYD)_

---

<a id="learning-objectives"></a>
### Learning Objectives 
- **Review** the limitations and shortcomings of OLS and the $R^2$ scoring function.
- **Define** when to use robust regression methods instead of OLS.
- **Demonstrate** how Theil-Sen, RANSAC, and Huber work and what they are optimizing for.
- **Explain** the advantages of using median absolute error as a scoring function.
- **Create** a scoring function that's appropriate for different business scenarios.


### Lesson Guide
- [Introduction](#introduction)
- [Review: Ordinary Least Squares and Squared Error Loss](#ols)
- [Robust Regression](#robust)
   - [Theil-Sen](#theilsen)
   - [RANSAC](#ransac)
   - [Huber](#huber)
- [Median Absolute Error](#mae)
- [Scenario-Specific Scoring Functions](#custom)
    - [Qiqi's Lemonade Stand](#qiqi-lemonade)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import seaborn as sns

% matplotlib inline

<a id="introduction"></a>


# Introduction

Congratulations! You've mastered linear regression. This is one of the most important skills a data scientist can have.

<img src="assets/kdnuggets-plot.jpg">

In this lesson, you'll learn some of the techniques data scientists use to leverage linear regression effectively in a commercial environment, including:

- A review of ordinary least squares.
- Robust regression.
- Useful metrics for evaluating a line.

This knowledge helps data scientists drive real value for an organization.


<a name="ols"></a>

## Review: Ordinary Least Squares ($R^2$)

---

So far, most examples have used ordinary least squares (OLS) regression to find a good linear model. With this technique, we look at all possible linear models and select one.

> When you look at what this model predicts and compare it to what was in the training data, there will be errors. For each point, you'll look at the square of the difference between what was predicted and what it was supposed to be, then sum up those squares. _The line you are looking for will be the one with the smallest sum._

Occasionally, this is a reasonable choice.

In [ ]:
# To illustrate some of these concepts, we'll make use of the following function to draw plots of numbers with lines through them to show the size of the error in each case.

def fancy_linear_plot(actuals, model, ax, include_error_bars=True):
    ax.scatter(x=actuals.index, y=actuals)
    predictions = pd.Series(index=actuals.index, data=model.predict(actuals.index.values.reshape(-1,1)))
    predictions.plot(c='red', ax=ax)
    squared_difference = 0
    differences = []
    for x in predictions.index:
        true_y = actuals.loc[x]
        predicted_y = predictions.loc[x]
        error_line = pd.Series(index=[x,x],data=[predicted_y,true_y])
        difference = true_y - predicted_y
        squared_difference += difference * difference
        differences.append("%.1f" % (difference * difference,))
        if include_error_bars:
            error_line.plot(ax=ax, linestyle='dashed', c='green')
            ax.annotate("error=%d" % (difference,),xy=(x+0.1,true_y-2 if true_y > predicted_y else true_y+2))
    if include_error_bars:
        title = "Sum of squares difference =\n%s\n = %d"  % (" + ".join(differences), squared_difference)
        ax.set_title(title)

In [ ]:
# With this example data, let's look at four potential linear models and what they look like.

example_data = pd.Series(
    data=[9,21,15,35,51,68,67,80],
    index=[1,2,3,4,5,6,7,8]
)


(fig, axes) = plt.subplots(nrows=2, ncols=2, figsize=(18,10))
plt.subplots_adjust(hspace=0.5)

lr1 = sklearn.linear_model.LinearRegression()
lr1.coef_ = np.array([20])
lr1.intercept_ = -40
fancy_linear_plot(example_data, lr1, axes[0][0])

lr2 = sklearn.linear_model.LinearRegression()
lr2.coef_ = np.array([10])
lr2.intercept_ = 0
fancy_linear_plot(example_data, lr2, axes[0][1])


lr3 = sklearn.linear_model.LinearRegression()
lr3.coef_ = np.array([3])
lr3.intercept_ = 30
fancy_linear_plot(example_data, lr3, axes[1][0])

lr4 = sklearn.linear_model.LinearRegression()
lr4.coef_ = np.array([-5])
lr4.intercept_ = 50
fancy_linear_plot(example_data, lr4, axes[1][1])

In [ ]:
# But, with ordinary least squares, we can get the one that has the smallest sum of squares.
(fig, ax) = plt.subplots(figsize=(9,5))
lr3 = sklearn.linear_model.LinearRegression()
lr3.fit(example_data.index.values.reshape(-1,1), example_data)
fancy_linear_plot(example_data, lr3, ax)

# Test Your Knowledge

1) True or false: The line generated by the ordinary least squares regression library in
   scikit-learn is the line that minimizes $R^2$.
   
2) True or false: Ordinary least squares regression only works when your data have
   two dimensions: one-dimensional $y$ and one-dimensional $X$.
   
3) True or false: The line found by minimizing $R^2$ isn't affected much by outliers.

## Why We Like $R^2$

- It's a unique answer.
- It's easy to calculate because we can use derivatives to create a formula to find it quickly.
- It makes sure that big mistakes are as small as possible. Simply making a small adjustment to the line to shrink the square of a big error can be worthwhile, even if it means losing accuracy on some small errors.

## The Problems With $R^2$

- **One outlier can ruin your whole model**. Ordinary least squares requires each outlier in one direction to have a corresponding quantity of outliers in the other direction (heteroskedasticity).

- This assumes that your $X$ values are measured with no error.
- **To a business, the dollar cost of a faulty prediction probably doesn't correspond** to the square of how wrong that prediction was. Typically, we want to **maximize the company's profits**, not minimize the squared error in our predictions.

Therefore, outside of specific theoritical situations, it is very rare for $R^2$ to be the right technique to use.

----

# What Should We Use *Instead* of Ordinary Least Squares?

- **Regularization**: This is one option when you want to find a simple model.

- **Robust regression**: This is a great option when your main challenge is finding a signal in a noisy data set.

Both of these methods are usually used in conjunction with **custom scoring functions**, especially when you know the cost to the business of the errors in your model. We'll be covering robust regression here.



<a name="#robust"></a>

----

## Robust Regression

Robust regression is a technique used to generate a linear model that's *the best* without being sensitive to outliers and noise. Some of its methods are so simple that they can be performed by hand, even on large data sets. Some, however, are so complex that they can only be done with modern computers.
They differ in how they define *the best linear model* and their results will rarely be the same as the least squares linear model. Let's look at an example where the least squares line is obviously incorrect.



In [ ]:
# First, draw the picture without any line through it.
noisy_example = pd.Series(
    data=[1,9,150,27,35,51,60,67,80],
    index=[0,1,2,3,4,5,6,7,8]
)
(fig, ax) = plt.subplots(figsize=(14,5), ncols=2)
ax[0].scatter(x=noisy_example.index, y=noisy_example)

# Create another chart with an ordinary least squares model overlay.
ols = sklearn.linear_model.LinearRegression()
ols.fit(noisy_example.index.values.reshape(-1,1), noisy_example)
fancy_linear_plot(noisy_example, ols, ax[1], include_error_bars=False)

### Class Exercise

1) In plain English, describe why the ordinary least squares model isn't effective in modeling this data set.
- Why doesn't minimizing the sum of the square of the error terms model this data well?
2) What would be a better metric? How could you compare two lines and decide which one fits the data better?



<a name="theilsen"></a>

----

## Theil-Sen

(Pronounced "Tail-Sen.")

The Theil-Sen method is simple and yet works surprisingly well.

1) Pick a pair of points at random.
2) Draw a line through them and note the gradient of the line.
3) Repeat Steps 1 and 2 some number of times (e.g. 25 times).
4) Sort the lines by gradient.
5) Choose the line with the median gradient.

(If you have more than two columns in your $X$ DataFrame, choose three points and draw a plane. If you have three columns, choose four points and create a hyperplane, and so on.)

**Theil-Sen's definition of best**: A linear model that isn't extreme; a "typical" linear model in which all of the other reasonable linear models are spread out evenly above and below the one that was chosen.



----

<img src="assets/theilsen.jpg" style="width: 400px; float: left; margin: 24px">

### Optional Group Activity

The best way to understand Theil-Sen is to do it by hand.

1) Clear a big space in the classroom. <br>
2) Create some numbered tokens. You can just tear up small pieces of paper and write distinct numbers on them. <br>
3) Start in one corner of the room, scattering the tokens on the floor in a vaguely straight line. Throw some tokens around randomly to act as outliers.
4) Use the handy random number generator in the next cell (or some dice, or your phones, etc) to choose a random pair of tokens. If you have some string and tape, you can stick a line on the floor that runs through those two points. If it's a surface that you can use chalk on, you could draw the line instead. If you only have stationery, you could place a pencil or pen on the floor in a line. <br>
5) Keep generating random pairs of points. Seven pairs should be enough, but you could do nine or 11. Sometimes you'll get crazy lines that are obviously wrong. <br>
6) Find the *middle* line out of the lines you drew. Did that produce a *reasonable* line through your data? Yes or no?



In [ ]:
from IPython.core.display import display, HTML
import random
number_of_tokens = 50
display(HTML(
    '<h1 style="color: red; font-size: 192px; margin: 48px">%d &amp; %d</h1>' % (
        random.randint(1,number_of_tokens),random.randint(1,number_of_tokens))))

<a name="ransac"></a>

----

## RANSAC

The RANSAC method is nearly as simple as Theil-Sen and can handle data sets that have a large number of outliers and a lot of random noise. Even if nearly 50 percent of your data set is noise, RANSAC can still find hidden trends that most other algorithms can't.

1) Pick a value or distance that you think is *close enough.*
2) Pick a random pair of points. Draw a line through them and then count the number of points that are *close enough* to the line.
3) Pick another pair of points, draw a line, and count the number of points that are *close enough* to the new line. If this line was better than the existing line, discard the existing line; otherwise, discard the new line.
4) Repeat Step 3 until you haven't seen an improvement in a long time. The line you have is your RANSAC best line. <br>

Like RANSAC, this works best in higher dimensions: Use triples of points forming a plane if you have two columns in your DataFrame.

RANSAC's definition of the **best linear model** is the one that's close to a lot of points; the one that models the data accurately *the most often*.



---
<img src="assets/ransac.jpg" style="float: left; width: 300px; margin-right: 24px">

### Optional Group Activity

This is similar to the Theil-Sen activity. If you still have markers on the floor, you can reuse them!

1) Find a convenient measuring device. If you have a ruler, that's good. Otherwise, you can use a shoe or anything else you can run along a line. <br>

2) Follow the RANSAC procedure: Pick a random pair of points and identify the line between them. One person carries the shoe/ruler/guide along the line; another person keeps count of the number of tokens it passes over. <br>

3) For this exercise, you'll usually have found a good line after 10 pairs of points. Beyond that, it's rare to find a better line (one where the shoe/ruler passes over more tokens). <br>

> In the picture on the left, the ruler passed over 18 tokens.

## Back to Code!

Let's see how these algorithms work on the data set we just explored where ordinary least squares performed so poorly.

The scikit-learn library includes functions for Theil-Sen and RANSAC.

In [ ]:
(fig, ax) = plt.subplots(figsize=(14,5), ncols=2)

ts = sklearn.linear_model.TheilSenRegressor()
ts.fit(noisy_example.index.values.reshape(-1,1), noisy_example)
fancy_linear_plot(noisy_example, ts, ax[0], include_error_bars=False)

ransac = sklearn.linear_model.RANSACRegressor()
ransac.fit(noisy_example.index.values.reshape(-1,1), noisy_example)
fancy_linear_plot(noisy_example, ransac.estimator_, ax[1], include_error_bars=False)

ax[0].annotate("Theil-Sen", (5,10), size=18)
ax[1].annotate("RANSAC", (5,10), size=18)

#### A Reminder

Theil-Sen and RANSAC have their own definitions for *the best linear model*. If you measure them with $R^2$, they will do quite badly!



In [ ]:
(fig, ax) = plt.subplots(figsize=(18,7), ncols=3)

fancy_linear_plot(noisy_example, ts, ax[0])
fancy_linear_plot(noisy_example, ransac, ax[1])
fancy_linear_plot(noisy_example, ols, ax[2])
ax[0].annotate("Theil-Sen", (5,10), size=18)
ax[1].annotate("RANSAC", (5,10), size=18)
ax[2].annotate("Least Squares", (4,10), size=18)

## Huber

The **Huber** regressor minimizes the sum of:
- The absolute value of the prediction error for predictions that were wrong by a large margin.
- The square of the value of the prediction error for predictions that were close.

This turns out to be very effective and is one of the most-used robust linear regressors. Outliers don't have disproportionate effects, but Huber will still try to minimize local errors.

In [ ]:
(fig, ax) = plt.subplots(figsize=(8,5), ncols=1)

huber = sklearn.linear_model.HuberRegressor()
huber.fit(noisy_example.index.values.reshape(-1,1), noisy_example)
fancy_linear_plot(noisy_example, huber, ax, include_error_bars=False)

ax.annotate("Huber", (6.5,10), size=18)

## Summary of Regressors

| **Regressor**          | **Handles Outliers...**   | **Useful When...**  |
|------------------------|------------|----------------------------------|
| Huber                  | Tolerant of unbalanced outliers.        | Usually the best one to try first. |
| Theil-Sen              | Unaffected if < 29 percent of data.     | Your $X$ values have errors and noise. |
| RANSAC                 | Unaffected if < 50 percent of data.     | Your $Y$ values have errors and noise. |
| Ordinary Least Squares | Only if normally distributed (rare).    | Avoiding large errors is important. |


## Test Your Knowledge

1) What's the difference between a robust regressor and a non-robust regressor?

2) What are the three robust regressors you've learned about today and how do they differ? What are the pros/cons of each?



# Scoring

How do we know whether or not a regressor has worked well? What do we mean by *best* line?

There are several ways of scoring a regressor:

- **$R^2$**: If you use this, then ordinary least squares will get you the best line.

- **Median absolute error**: If this is your measure, you will cope with large numbers of outliers.

- **Dollar value loss**: If this is your measure, your organization will thank you.


<font size=-5>$^*$ YMMV</font>

## Median Absolute Error

Scikit-learn includes several different out-of-the-box scoring functions for regressors and more than a dozen scoring functions for classifiers.

**Median absolute error** is commonly used for **noisy data**. You will have some points that you predicted well (so the absolute value of the error on those points will be quite small) and some that you predicted badly (so the absolute value of the error on those points will be quite large). What's the median of those points? Fifty percent of your errors fall below this number.

This (and many other) metric functions are defined in `sklearn.metrics.`, but for many you can use a shorthand string (e.g., `'median_absolute_error'`) anywhere that a metric is passed as an argument.



In [ ]:
# It doesn't normally make sense to measure how well a regressor predicts on data it was given for training, but here we'll do it anyway. 

import sklearn.metrics
def score(model):
    predictions = model.predict(noisy_example.index.values.reshape(-1,1))
    return sklearn.metrics.median_absolute_error(noisy_example, predictions)

print("OLS:",score(ols))
print("Theil-Sen:", score(ts))
print("RANSAC:", score(ransac))
print("Huber:", score(huber))

I.e., for Huber, half of the points were predicted within 1.82 of the actual value. Or, to consider it negatively, for half of the points, ordinary least squares was more than 18.3333 away from the actual value.



<a name='custom'></a>

----

# Scenario-Specific Scoring Functions

$R^2$ is reasonably common (and often misapplied). *Median absolute error* is commonly used, as well. But **the most common scoring method** measures the error in **dollars** (or whatever currency your business uses).

In most organizations, your model will produce a prediction that's used to help with decision-making, and there will be costs associated an error. Often, those costs will be **asymmetric** (overestimating might be more expensive than underestimating).

In some cases, you may work for a startup or organization that isn't very concerned with dollar costs. In this case, the right metric may involve monthly users, sign-ups, or other growth-relevant metrics.


----

<a name="qiqi-lemonade"></a>

## Lemonade Stand

<img src="assets/lemonade-stand.jpg" style="width: 200px; float: left; margin: 20px">

Let's say it costs Qiqi 3 cents to buy the sugar, lemons, and cups necessary to make one glass of lemonade. She has to spend this money to buy the ingredients upfront at the beginning of the day - and can't buy more until the next day if she runs out.

She pays her brother a fixed fee of \$2 to make the mixture, regardless of how many cups she sells each day. She charges customers 50 cents.

Qiqi has access to data showing that there is a relationship between the temperature and humidity and the number of customers who will want to buy a cup of lemonade. She is trying to choose between several linear models.

**Question**: What would be good scoring function to help Qiqi evaluate which model to use?



**Answer:**

First, discard a situation where Qiqi decides not to be in business at all! Next, the \$2 to her brother is a fixed cost; therefore, we can ignore it.

In the test portion of the database, let $X_p$ be the predicted number of customers and $X_a$ be the actual number of customers for a particular day.

- If $X_p > X_a$, then Qiqi would lose $0.03 * (X_p - X_a)$ in wasted materials if she had followed that model.

- If $X_a > X_p$, then Qiqi would lose $0.50 * (X_a - X_p)$ in lost sales if she had followed that model.

- If $X_p = X_a$, then Qiqi would waste nothing and lose no sales.

The closer the score is to zero, the better.



In [ ]:
def lemonade_scorer(estimator, X, y):
    predictions = list(estimator.predict(X))
    y = list(y)
    score = 0.0
    for i in range(len(y)):
        if predictions[i] > y[i]:
            score -= 0.03 * int(predictions[i] - y[i])
        elif predictions[i] < y[i]:
            score -= 0.5 * int(y[i] - predictions[i])
        else:
            score -= 0.0
    return score

In [ ]:
lemonade_stand = pd.read_csv('datasets/lemonade-stand.csv', index_col=0)
lemonade_stand.head()

In [ ]:
# There is a relationship between temperature and sales.
# There is a (weaker) relationship between increased humidity and decreased sales, which could have
# something to do with whether or not it was raining!
# However, there are still about five days in which sales were much lower than would have been expected.
sns.pairplot(lemonade_stand)

In [ ]:
# Split the data into training and testing sets. If we train on the training set, we can then get an estimate of the dollar cost of the model by trying it out on the testing set.
(Xtrain, Xtest, Ytrain, Ytest) = sklearn.model_selection.train_test_split(lemonade_stand[['Max_Temp_C','humidity']],
                                                                         lemonade_stand.sales)

In [ ]:
# Let's try out our four different linear models: OLS, Theil-Sen, RANSAC, and Huber.
lemonade_ols = sklearn.linear_model.LinearRegression()
lemonade_ts = sklearn.linear_model.TheilSenRegressor()
lemonade_ransac = sklearn.linear_model.RANSACRegressor()
lemonade_huber = sklearn.linear_model.HuberRegressor()

In [ ]:
# Call .fit() on each model using the training data.
lemonade_ols.fit(Xtrain, Ytrain)
lemonade_ts.fit(Xtrain, Ytrain)
lemonade_ransac.fit(Xtrain, Ytrain)
lemonade_huber.fit(Xtrain, Ytrain)

In [ ]:
# Use Matplotlib to help visualize what's going on.
(fig, axes) = plt.subplots(nrows=2, ncols=2, figsize=(16,10))
plt.subplots_adjust(hspace=0.5)

def show_predictions(ax, regressor, name):
    predictions = y=regressor.predict(Xtest)
    colouring = np.where(predictions < Ytest, 'red', 'green')
    ax.scatter(x=Ytest, y=predictions, c=colouring)
    ax.set_title(name + " (Predictions vs Actual)\n$%.2f loss" % (lemonade_scorer(regressor, Xtest, Ytest),))
    ax.plot([Ytest.min(),Ytest.max()],[Ytest.min(),Ytest.max()], linestyle='dotted', c='#e0e0ff')
    ax.set_xlabel("Actual")
    ax.set_ylabel(name + " predictions")
    ax.annotate("Predicted too many sales\n(inexpensive mistake)", xy=(Ytest.min()+1, Ytest.max()-2), color='green')
    ax.annotate("Predicted too few sales\n(expensive mistake)", xy=(Ytest.max()-9,Ytest.min()+3), color='red')
    
show_predictions(axes[0][0], lemonade_ols, 'Least Squares')
show_predictions(axes[0][1], lemonade_ts, 'Theil-Sen')
show_predictions(axes[1][0], lemonade_ransac, 'RANSAC')
show_predictions(axes[1][1], lemonade_huber, 'Huber')

To put it plainly, what's happening here is that the least squares regression is trying hard to minimize the error from the five or so times that sales were far lower than would be expected based on temperature and humidity.

But, saving a few cents on the input materials is overwhelmed by the losses incurred from not making high-enough predictions for typical days.



# Recap

## Methods

- If you want to generate a **quick approximation** for a good line through a data set, you can use Theil-Sen with or without a computer.

- If you believe you have a **outliers in your data** (which are common) that don't affect the whole, then use one of the robust regression techniques: Huber, RANSAC, or Theil-Sen.


## Scoring

- **Ordinary least squares** will try to minimize large errors; while it depends on the scenario, this is usually not the right thing to minimize.

- Minimizing the **median absolute error** is more effective.
  
- In most commercial scenarios, your goal is rarely accuracy. Make sure your scoring functions match the errors that your organization wants to minimize.




# Additional Resources

- Ordinary least squares is related to Pearson's coefficient in the same way that Theil-Sen is related to Kendall's
$\tau$ coefficient; OLS is parametric, whereas Theil-Sen is non-parametric. There have been various attempts to remedy some of Theil-Sen's deficiencies. CRAN's paper on [Deming regression](https://cran.r-project.org/web/packages/deming/vignettes/deming.pdf]) discusses these topics.
- [Linear Valuation Without OLS: The Theil-Sen Estimation Approach](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2276927) — financial and accounting data are easier to predict with Theil-Sen than OLS.
- [Daniel Wedge wrote a *song* about RANSAC](http://danielwedge.com/ransac/) — he also has a song about $e$.
- Watch this YouTube video on [robust regression with Huber weighting](https://www.youtube.com/watch?v=0drbiDPCuYQ).

